In [1]:
# Process BES Panel for Wave 8

## WORK IN PROGRESS - run this on all updated BES waves

# Assign types to each column variables (mostly automatic - but there's room for manual editing)
# Modifies categories (harmoning options, harmonising order, dropping weasel terms)
# Creates:
# var_type (types for column variables)
# BES_numeric (fixed version with only useful columns all transformed into numeric values)
    # non-numeric categorical values -> index number of category
    # floats -> float
    # numeric categories (i.e. income intervals/number of children) -> numeric
# cat_dictionary
    # so you can get the text categories back from the index values as and when it's necessary
# BES_non_numeric
    # basically everything else! (mostly meta-data - weights)

# LIVE: Run all cells -> ~20s

In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import gc
import re

from itertools import cycle
from IPython.display import display
import pickle, os

import seaborn as sns

# BES_data_folder = "../BES_analysis_data/"
# BES_code_folder = "../BES_analysis_code/"


In [3]:
# you should clone this git to this subdirectory (in some directory - I call it BES_analysis - doesn't matter though)

if os.getcwd().split(os.sep)[-1] != 'BES_analysis_code':
    raise Exception("Stop! You're in the wrong directory - should be in 'BES_analysis_code'")

BES_code_folder   = "../BES_analysis_code/" # we should be here!
BES_small_data_files = BES_code_folder + "small data files" + os.sep
if not os.path.exists( BES_small_data_files ):
    os.makedirs( BES_small_data_files )

# we should create these if they don't already exist
BES_data_folder   = "../BES_analysis_data/"
if not os.path.exists( BES_data_folder ):
    os.makedirs( BES_data_folder )

BES_output_folder = "../BES_analysis_output/"
if not os.path.exists( BES_output_folder ):
    os.makedirs( BES_output_folder )

In [4]:
encoding = "ISO-8859-1"

In [5]:
import Jupyter_module_loader
import Untitled5
from Untitled5 import process_dataset

importing Jupyter notebook from Untitled5.ipynb


In [6]:
dataset_name = 'W5_comb'

BES_file_manifest = pd.read_csv( BES_small_data_files + "BES_file_manifest.csv" )

In [7]:
process_dataset(dataset_name)

In [8]:
# TEST CODE
# Checking to see if different column names have same question set but are assigned different types

In [9]:
# # Test to see if pruning "_wave" leads to differing variable categories
# for key in var_cat_dict_pruned.keys():
#     for val in var_cat_dict_pruned[key]:
#         for other_key in var_cat_dict_pruned.keys():
#             if key==other_key: # don't check same category
#                 continue
#             if val in var_cat_dict_pruned[other_key]:
#                 print("problem: {0} {1} {2}".format(key, val, other_key) )
            
# # mixing 1 and 5 is fine, because 5 (mis-ordered ordinal) is turned into 1 (correctly ordered ordinal)

In [10]:
# # Test to see if further pruning "_num" leads to differing variable categories
# for key in var_cat_dict_pruned_2.keys():
#     for val in var_cat_dict_pruned_2[key]:
#         for other_key in var_cat_dict_pruned_2.keys():
#             if key==other_key: # don't check same category
#                 continue
#             if val in var_cat_dict_pruned_2[other_key]:
#                 print("problem: {0} {1} {2}".format(key, val, other_key) )
            
                

In [11]:
var_type_all_datasets = pd.DataFrame(columns = ["dataset_name","dtype","type","pruned","original_cat_list",
                                   "renamed_cat_list","reordered_cat_list","final_cat_list",
                                   "dataset_specific_hardcoded_fix",
                                   "numerical_dont_knows",
                                   "weasel_words","typos" ] )

for dataset_name in [x for x in BES_file_manifest["Name"] if not ("strings" in x)]:
    print(dataset_name)

    data_subfolder = BES_data_folder + dataset_name + os.sep
    
#     var_type = pd.read_hdf( data_subfolder + "var_type.hdf", "var_type")
    var_type = pd.read_csv( data_subfolder + "var_type.csv", encoding = encoding )
    
    var_type_all_datasets = var_type_all_datasets.append(var_type)

var_type_all_datasets.reset_index(inplace = True)
var_type_all_datasets.rename(columns = {'index':'variable_name'},inplace = True)    

var_type_all_datasets.to_csv( BES_data_folder + "var_type_all_datasets.csv", encoding = encoding)

W13_only
W13_comb
W12_only
W11_only
W10_comb
W10_only
W9_comb
W9_only
W8_only
W8_comb
W7_only
W7_comb
W6_only
W6_comb
W5_comb
W5_only
W4_only
W4_comb
W3_only
W3_comb
W2_only
W1_only


In [12]:
# ## COLUMNS THAT EITHER LACK ALL DATA OR HAVE ACTUAL ERRORS
# # check back on these periodically - one assumes they will get fixed!
# # maybe tell them about them so that they can?

# # {'changeIssue1W9', 'conLeaderLikeW9'}
# # these variables appear to have disappeared! Fixed in an updated version?

# ignore_list = ['whichPartiesHelped_99W6',
#                'partyContactGrnW1',
#                'partyContactGrnW2',
#                'partyContactGrnW3',
#                'reasonNotRegistered_noneW2',               
#                'reasonNotRegistered_noneW3',
#                'reasonNotRegistered_noneW4',
#                'reasonNotRegistered_noneW6',
#                'reasonNotRegistered_noneW7',
#                'reasonNotRegistered_noneW8',
#                'reasonNotRegistered_none',
#                'partyContactSNPW1',
#                'partyContactSNPW2',
#                "locusControlW9",
#                "generalElecCertaintyW1", # wave 10 forwards
#                "generalElecCertaintyW2",
#                "generalElecCertaintyW3",
#                "londonMayorVoteW7",
#                "fatherNumEmployeesW4",
#                "motherNumEmployeesW4",
#                "profile_pcon_2010_newW3", # W3_comb: this is parl. constit. ... but by number!
#                "euroElectionVoteYoungW2", # W3_comb: all NaNs!
#                "profile_GOR_pdlW4", # W4_comb: misnamed selection, probably fixable 
#                "participation_111W5", ### -->
#                "sharedContentOnline_111W5",
#                "sharedContentOnline_99W5", ### <-- W5_comb "Got a lot worse|Got a little worse" doesn't look right (indicator vars?)
#                "csplScotRefW3", ### W5_comb: "North East" - just broken!
#               ]

# #- approveEUW2 'Strongly disapprove|Disapprove|Don't know' - should be "approve|disapprove|don't know"??? NOT SURE (distribution weird)
# #- whichPartiesHelped_99W6 - answer set = ["No"]
# #- partyContactGrnW1 ... reasonNotRegistered_noneW8 answer set = ["No", "Don't know"]
# # -partyContactSNPW1, partyContactSNPW2 - answer set = ["Don't know"]
# # -changeIssue1W9|conLeaderLikeW9|locusControlW9 - answer set = ["No formal qualifications"]

In [13]:
# var_type_all_datasets = pd.read_csv( BES_data_folder + "var_type_all_datasets.csv", encoding = encoding )

# BES_file_manifest = pd.read_csv( BES_small_data_files + "BES_file_manifest.csv" )
# ign_out = pd.DataFrame(columns = ["dataset_name","variable_name","dtype","cat_all_strings","cat_list"])
# ign_ds = var_type_all_datasets[ var_type_all_datasets["variable_name"].apply(lambda x: x in ignore_list) ]["dataset_name"].unique()

# for dataset_name in ign_ds:

#     manifest = BES_file_manifest[ BES_file_manifest["Name"] == dataset_name ]

#     filename = manifest["Stata_Filename"].values[0]

#     data_subfolder = BES_data_folder + dataset_name + os.sep

#     BES_Panel = pd.read_stata( data_subfolder + filename)

#     w13_igns = var_type_all_datasets[ var_type_all_datasets["variable_name"].apply(lambda x: x in ignore_list) ]\
#         [var_type_all_datasets["dataset_name"]==dataset_name]


#     for var in w13_igns["variable_name"].values:
        
#         dtype = BES_Panel[ var ].dtypes.name
#         if (dtype == 'category'):
#             cat_all_strings = np.all([isinstance(x,str) for x in BES_Panel[ var ].cat.categories])
#             if cat_all_strings:
#                 cat_list = "|".join(BES_Panel[ var ].cat.categories)
#                 print(var,":" ,'"'+cat_list+'"' )
#             else:
#                 cat_list = BES_Panel[ var ].cat.categories
#                 print(var,":" ,cat_list )
#         else:
#             cat_all_strings = np.nan
#             cat_list = (BES_Panel[ var ].dropna().unique())
#             print(var,":" , cat_list )
            
#         temp_row = pd.DataFrame([[dataset_name,var,dtype,cat_all_strings,cat_list]],
#             columns = ["dataset_name","variable_name","dtype","cat_all_strings","cat_list"])            
#         ign_out = ign_out.append( temp_row )      

# ign_out.reset_index( drop = True , inplace = True )        
# ign_out.to_csv( BES_data_folder + "ignore_out.csv", encoding = encoding )

In [14]:
# ign_out = pd.read_csv( BES_data_folder + "ignore_out.csv", encoding = encoding )

# ign_out_dataset_list = pd.DataFrame(columns = ["dataset_list","variable_name","dtype","cat_all_strings","cat_list"])
# newdf_index = 0
# for col in ign_out["variable_name"].unique():
#     if re.match("soc2010|v1|RandomIDW1|mapNames(W[0-9]+)?" ,col) is not None:
#         continue
#     elif re.match( "pano(W[0-9]+)?|electoratepcon|[a-zA-Z]+10pcon" , col ) is not None:
#         continue
#     elif re.match( "approveEUW2|selfNumEmployeesW6W7W8W9|selfNumEmployeesLastW6W7W8W9" , col ) is not None:
#         continue

#     indices = ign_out[ ign_out["variable_name"]==col ].index
#     dataset_list = []
#     dataset_list.append( ign_out.loc[ indices[0] ]["dataset_name"] )  
    
#     cat_list        = ign_out.loc[ indices[0] ]["cat_list"]
#     dtype           = ign_out.loc[ indices[0] ]["dtype"]
#     cat_all_strings = ign_out.loc[ indices[0] ]["cat_all_strings"]    
    
#     if len(indices) > 1:

#         for index in indices[1:len(indices)]:
# #             if ( not np.all([i==j for i, j in zip(ign_out.loc[ index ]["cat_list"], cat_list)]) ):
# #                 print( col, cat_list, ign_out.loc[ index ]["cat_list"] )
#             if ( ign_out.loc[ index ]["dtype"] != dtype ):
#                 print( col, dtype   , ign_out.loc[ index ]["dtype"] )
#             if ( not np.isnan(ign_out.loc[ index ]["cat_all_strings"]) and\
#                 (ign_out.loc[ index ]["cat_all_strings"] != cat_all_strings) ):
#                 print( col, cat_all_strings   , ign_out.loc[ index ]["cat_all_strings"] )
#                 dataset_list.append( ign_out.loc[ index ]["dataset_name"] )

#     ign_out_dataset_list.loc[newdf_index] = [dataset_list, col, dtype, cat_all_strings, cat_list ]
#     newdf_index = newdf_index + 1
        
        
# ign_out_dataset_list.sort_values(by="variable_name").to_csv( BES_data_folder + "ignore_out_stacked_by_dataset.csv", encoding = encoding )

In [429]:
%%time
dataset_name = "W7_comb"

manifest = BES_file_manifest[ BES_file_manifest["Name"] == dataset_name ]

filename = manifest["Stata_Filename"].values[0]

data_subfolder = BES_data_folder + dataset_name + os.sep
# global BES_Panel_W7comb
BES_Panel_W7comb = pd.read_stata( data_subfolder + filename)

Wall time: 4min 57s


In [439]:
%%time
dataset_name = "W6_comb"

manifest = BES_file_manifest[ BES_file_manifest["Name"] == dataset_name ]

filename = manifest["Stata_Filename"].values[0]

data_subfolder = BES_data_folder + dataset_name + os.sep
global BES_Panel
BES_Panel = pd.read_stata( data_subfolder + filename)


Wall time: 3min 4s


['Should definitely be illegal',
 'Should probably be illegal',
 'Should probably be legal',
 'Should definitely be legal',
 '5.0',
 "Don't know"]

In [436]:
# W7_only
col = "noDependentsInHousehold"
BES_Panel[ col ].value_counts()

Yes    22778
No      7895
2.0        7
Name: noDependentsInHousehold, dtype: int64

In [434]:
# W7_only
col = "noDependentsInHousehold"
BES_Panel[ col ].value_counts()

Yes    22986
No      7876
Name: noDependentsInHousehold, dtype: int64

In [417]:
BES_Panel[ col ].value_counts().values.sum()

30027

In [430]:
# W7_comb
col = "certaintyEUGreenW2"
BES_Panel_W7comb[ col ].value_counts()
# [x for x in BES_Panel_only.columns if "certaintyEU" in x]

Don't know            856
Somewhat certain      836
Not at all certain    564
Very certain          461
Name: certaintyEUGreenW2, dtype: int64

Should definitely be illegal    9718
Should probably be illegal      7417
Should definitely be legal      5267
Should probably be legal        4493
Don't know                      1615
5.0                             1517
Name: zeroHourContractW6, dtype: int64

In [411]:
# W6_only
col = "zeroHourContract"
BES_Panel[ col ].value_counts()

Should definitely be illegal    9733
Should probably be illegal      7426
Should definitely be legal      5278
Should probably be legal        4501
Don't know                      1615
Name: zeroHourContract, dtype: int64

In [409]:
# col = "housing"
BES_Panel[ col ].cat.categories


Index(['None/ No leader',   'David Cameron',     'Ed Miliband',
            'Nick Clegg', 'Nicola Sturgeon',     'Leanne Wood',
          'Nigel Farage', 'Natalie Bennett',               222,
            'Don't know'],
      dtype='object')

In [386]:
BES_Panel[col].cat.categories

Index([                                                                      'Own  outright',
                                                                      'Own  with a mortgage',
       'Own (part-own)  through shared ownership scheme (i.e. pay part mortgage, part rent)',
                                                             'Rent  from a private landlord',
                                                             'Rent  from my local authority',
              'Neither  I live with my parents, family or friends but pay some rent to them',
                              'Neither  I live rent-free with my parents, family or friends',
                                                                                      'Other',
                                                                                         9999],
      dtype='object')

In [395]:
BES_Panel[col].cat.rename_categories( BES_Panel[col].cat.categories.map(str),inplace=True )
join_list = "|".join(  BES_Panel[col].cat.categories )
join_list

'Own \x96 outright|Own \x96 with a mortgage|Own (part-own) \x96 through shared ownership scheme (i.e. pay part mortgage, part rent)|Rent \x96 from a private landlord|Rent \x96 from my local authority|Neither \x96 I live with my parents, family or friends but pay some rent to them|Neither \x96 I live rent-free with my parents, family or friends|Other|9999'

In [403]:
'Own \x96 outright|Own \x96 with a mortgage|Own (part-own) \x96 through shared ownership scheme (i.e. pay part mortgage, part rent)|Rent \x96 from a private landlord|Rent \x96 from my local authority|Neither \x96 I live with my parents, family or friends but pay some rent to them|Neither \x96 I live rent-free with my parents, family or friends|Other|9999'.split("|")

['Own \x96 outright',
 'Own \x96 with a mortgage',
 'Own (part-own) \x96 through shared ownership scheme (i.e. pay part mortgage, part rent)',
 'Rent \x96 from a private landlord',
 'Rent \x96 from my local authority',
 'Neither \x96 I live with my parents, family or friends but pay some rent to them',
 'Neither \x96 I live rent-free with my parents, family or friends',
 'Other',
 '9999']

In [396]:
fixed_cat_string = fix_a_hat_chars( join_list )
fixed_cat_string

['Own - outright',
 'Own - with a mortgage',
 'Own (part-own) - through shared ownership scheme (i.e. pay part mortgage, part rent)',
 'Rent - from a private landlord',
 'Rent - from my local authority',
 'Neither - I live with my parents, family or friends but pay some rent to them',
 'Neither - I live rent-free with my parents, family or friends',
 'Other',
 '9999']

In [400]:
"|".join( fixed_cat_string )

'Own - outright|Own - with a mortgage|Own (part-own) - through shared ownership scheme (i.e. pay part mortgage, part rent)|Rent - from a private landlord|Rent - from my local authority|Neither - I live with my parents, family or friends but pay some rent to them|Neither - I live rent-free with my parents, family or friends|Other|9999'

In [384]:
# s.cat.rename_categories([1,2,3])
# EUContactRemainConW8|EUContactRemainLabW8|EUContactRemainLDW8|
# EUContactRemainSNPW8|EUContactRemainPCW8|EUContactRemainUKIPW8|
# EUContactRemainGreenW8|EUContactRemainOthW8|EUContactRemainNoneW8|
# EUContactRemainDKW8|EUContactLeaveConW8|EUContactLeaveLabW8|
# EUContactLeaveLDW8|EUContactLeaveSNPW8|EUContactLeavePCW8|
# EUContactLeaveUKIPW8|EUContactLeaveGreenW8|EUContactLeaveOthW8|
# EUContactLeaveNoneW8|EUContactLeaveDKW8

# pattern match "EUContact*****W8"
# debateOneWatchW8|debateTwoWatchW8

# "1.0|2.0|99.0" -> 

# euRefVoteSqueezeW7 "Will not vote|Yes - Leave|Yes - Remain|No - not decided"
#    -> Stay/remain in the EU|Leave the EU|I would/will not vote|Don't know
#    HMM - RENAME AND REORDER!

# miieuW7
# "Issue stated|Nothing|Don't know" -> "Issue stated|None|Don't know"
# MIIEUW8
# "1.0|Nothing|Don't know" -> "Issue stated|None|Don't know"
# partyIdEUW7|partyIdEUW8
# "Mainly leave|Mainly remain|Fairly evenly split|Don't know" -> "Mainly remain|Fairly evenly divided|Mainly leave|Don't know"
#    HMM - RENAME AND REORDER!

# 1. campaignVisionYesW3|campaignVisionNoW3, govtNatSecuritySuccessW4
# Very unsuccessful|Fairly unsuccessful|Neither successful nor unsuccessful|Fairly successful|Very successful|Don't know
# Very unsuccessful|Somewhat unsuccessful|Neither successful or unsuccessful|Somewhat successful|Very successful|Don't know

# Fairly <-> Somewhat

# 2. euroTurnoutW1, scotReferendumTurnoutW1|scotReferendumTurnoutW2|welshTurnoutW7|scotTurnoutW7, turnoutUKGeneralW1|turnoutUKGeneralW2|turnoutUKGeneralW3|turnoutUKGeneralW4|turnoutUKGeneralW5|euRefTurnoutW7|euRefTurnoutW8
# Very unlikely that I vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I vote|Don't know
# Very unlikely that I would vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I would vote|Don't know
# There are not local elections in my area
    #|Very unlikely that I will vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I will vote|Don't know
# Very unlikely that I will vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I will vote|Don't know

# "Very unlikely that I vote", "Very unlikely that I would vote" ->  "Very unlikely that I will vote" 

rename_cat_dict = {"North East|North West": [ "No", "Yes" ],
                   "1.0|2.0|99.0": ["No", "Yes", "99.0"],
                   "Will not vote|Yes - Leave|Yes - Remain|No - not decided":
                       ['I would/will not vote', 'Leave the EU','Stay in the EU', "Don't know"], 
                   "Stay/remain in the EU|Leave the EU|I would/will not vote|Don't know":
                       ['Stay in the EU','Leave the EU',  'I would/will not vote', "Don't know"],   # euRefVote    
                   "Stay/remain in the EU|Leave the EU|Don't know":
                       ['Stay in the EU','Leave the EU', "Don't know"],   # profile_eurefvote                    
                   "Issue stated|Nothing|Don't know":  ['Issue stated', 'None', "Don't know"],
                   "1.0|Nothing|Don't know":           ['Issue stated', 'None', "Don't know"],
                   "a|b|C1|C2|d|e|Refused|Unknown" : ['A', 'B', 'C1', 'C2', 'D', 'E', 'Refused', 'Unknown'],
                   "Mainly leave|Mainly remain|Fairly evenly split|Don't know":
                       ['Mainly leave','Mainly remain', 'Fairly evenly divided', "Don't know"],
                   "Very unsuccessful|Somewhat unsuccessful|Neither successful or unsuccessful|Somewhat successful|Very successful|Don't know":
                       ['Very unsuccessful', 'Fairly unsuccessful', 'Neither successful nor unsuccessful',
                        'Fairly successful', 'Very successful', "Don't know"],
                   "Very unlikely that I vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I vote|Don't know":
                       ['Very unlikely that I will vote', 'Fairly unlikely', 'Neither likely nor unlikely',
                        'Fairly likely', 'Very likely that I will vote', "Don't know"],
                   "Very unlikely that I would vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I would vote|Don't know":
                       ['Very unlikely that I will vote', 'Fairly unlikely', 'Neither likely nor unlikely',
                        'Fairly likely', 'Very likely that I will vote', "Don't know"],
                   "No, did not vote|Yes, voted|3.0|4.0|5.0|Don't know":   
                       ["Very unlikely that I will vote", "Fairly unlikely", 'Neither likely nor unlikely',
                        "Fairly likely", "Very likely that I will vote", "Don't know"], #londonTurnoutW7
                   'No, I do not regard myself as belonging to any particular religion.|Yes - Church of England/Anglican/Episcopal|Yes - Roman Catholic|Yes - Presbyterian/Church of Scotland|Yes - Methodist|Yes - Baptist|Yes - United Reformed Church|Yes - Free Presbyterian|Yes - Brethren|Yes - Judaism|Yes - Hinduism|Yes - Islam|Yes - Sikhism|Yes - Buddhism|Yes - Other|Prefer not to say|Yes Orthodox Christian|Yes - Pentecostal (e.g. Assemblies of God, Elim Pentecostal Church, New Testament Church of God, Redeemed Christian Chur|Yes - Evangelical independent/non-denominational (e.g. FIEC, Pioneer, Vineyard, Newfrontiers)':
                       ["No, I do not regard myself as belonging to any particular religion.","Yes - Church of England/Anglican/Episcopal",
                        "Yes - Roman Catholic","Yes - Presbyterian/Church of Scotland","Yes - Methodist","Yes - Baptist",
                        "Yes - United Reformed Church","Yes - Free Presbyterian","Yes - Brethren","Yes - Judaism","Yes - Hinduism",
                        "Yes - Islam","Yes - Sikhism","Yes - Buddhism","Yes - Other","Prefer not to say","Yes - Orthodox Christian",
                        "Yes - Pentecostal","Yes - Evangelical /independent/non-denominational"], #xprofile_religionW10
                   'No, I do not regard myself as belonging to any particular religion.|Yes - Church of England/Anglican/Episcopal|Yes - Roman Catholic|Yes - Presbyterian/Church of Scotland|Yes - Methodist|Yes - Baptist|Yes - United Reformed Church|Yes - Free Presbyterian|Yes - Brethren|Yes - Judaism|Yes - Hinduism|Yes - Islam|Yes - Sikhism|Yes - Buddhism|Yes - Other|Prefer not to say|Yes - Orthodox Christian|Yes - Pentecostal (e.g. Assemblies of God, Elim Pentecostal Church, New Testament Church of God, Redeemed Christian Chur|Yes - Evangelical - independent/non-denominational (e.g. FIEC, Pioneer, Vineyard, Newfrontiers)':
                       ["No, I do not regard myself as belonging to any particular religion.","Yes - Church of England/Anglican/Episcopal",
                        "Yes - Roman Catholic","Yes - Presbyterian/Church of Scotland","Yes - Methodist","Yes - Baptist",
                        "Yes - United Reformed Church","Yes - Free Presbyterian","Yes - Brethren","Yes - Judaism","Yes - Hinduism",
                        "Yes - Islam","Yes - Sikhism","Yes - Buddhism","Yes - Other","Prefer not to say","Yes - Orthodox Christian",
                        "Yes - Pentecostal","Yes - Evangelical /independent/non-denominational"], #xprofile_religionW10                   
                   'Own - outright|Own - with a mortgage|Own (part-own) - through shared ownership scheme (i.e. pay part mortgage, part rent)|Rent - from a private landlord|Rent - from my local authority|Rent - from a housing association|Neither - I live with my parents, family or friends but pay some rent to them|Neither - I live rent-free with my parents, family or friends|Other|9999':
                       [ 'Own outright',
                         'Own with a mortgage',
                         'Own (part-own) through shared ownership scheme (i.e. pay part mortgage, part rent)',
                         'Rent from a private landlord',
                         'Rent from my local authority',
                         'Rent from a housing association',
                         'Neither I live with my parents, family or friends but pay some rent to them',
                         'Neither I live rent-free with my parents, family or friends',
                         'Other',
                         '9999'], #profile_house_tenureW11|profile_house_tenureW12|profile_house_tenureW13
                   "I voted 'No' (Scotland should not be an independent country)|I voted 'Yes' (Scotland should be an independent country)|111.0|Don't know":
                       ["No","Yes","Did not vote","Don't know"], # referendumrecall
                   "Voted Yes|Voted No|Did not vote|Can't remember":
                       ["Yes","No","Did not vote","Don't know"], # scotRefVoteW4_
                   "No|Yes|3.0|Don't know":
                       ["No","Yes","Did not vote","Don't know"], # regretsIHaveAFewEUW10|regretsIHaveAFewEUW11   
                   "No|Yes|3|Don't know":
                       ["No","Yes","Did not vote","Don't know"], # regretsIHaveAFewEU W11_only|regretsIHaveAFew W10_only 
                   "Professional or higher technical work - work that requires at least degree-level qualifications (e.g. doctor, accountant|Manager or Senior Administrator (e.g. company director, finance manager, personnel manager, senior sales manager, senior|Clerical (e.g. clerk, secretary)|Sales or Services (e.g. commercial traveller, shop assistant, nursery nurse, care assistant, paramedic)|Foreman or Supervisor of Other Workers (e.g building site foreman, supervisor of cleaning workers)|Skilled Manual Work (e.g. plumber, electrician, fitter)|Semi-Skilled or Unskilled Manual Work (e.g. machine operator, assembler, postman, waitress, cleaner, labourer, driver, b|Other|Have never worked":
                       ['Professional or higher technical work / higher managerial - work that requires at least degree-level qualifications (e.g',
                        'Manager or Senior Administrator / intermediate managerial / professional (e.g. company director, finance manager, person',
                        'Clerical/junior managerial/professional/administrator (e.g. office worker, student doctor, sales person, clerk, secretar',
                        'Sales or Services (e.g. commercial traveller, shop assistant, nursery nurse, care assistant, paramedic)',
                        'Foreman or Supervisor of Other Workers (e.g. building site foreman, supervisor of cleaning workers)',
                        'Skilled Manual Work (e.g. plumber, electrician, fitter)',
                        'Semi-Skilled or Unskilled Manual Work (e.g. machine operator, assembler, postman, waitress, cleaner, labourer, driver, b',
                        'Other',
                        'Have never worked'], # work_type -> profile_work_typeW7
                   "No formal qualifications|Youth training certificate/skillseekers|Recognised trade apprenticeship completed|Clerical and commercial|City & Guilds certificate|City & Guilds certificate - advanced|onc|CSE grades 2-5|CSE grade 1, GCE O level, GCSE, School Certificate|Scottish Ordinary/ Lower Certificate|GCE A level or Higher Certificate|Scottish Higher Certificate|Nursing qualification (eg SEN, SRN, SCM, RGN)|Teaching qualification (not degree)|University diploma|University or CNAA first degree (eg BA, B.Sc, B.Ed)|University or CNAA higher degree (eg M.Sc, Ph.D)|Other technical, professional or higher qualification|Don't know|Prefer not to say":
                       ['No formal qualifications','Youth training certificate/skillseekers','Recognised trade apprenticeship completed',
                        'Clerical and commercial','City and Guild certificate','City and Guild certificate - advanced','onc','CSE grades 2-5',
                        'CSE grade 1, GCE O level, GCSE, School Certificate','Scottish Ordinary/ Lower Certificate','GCE A level or Higher Certificate',
                        'Scottish Higher Certificate','Nursing qualification (eg SEN, SRN, SCM, RGN)','Teaching qualification (not degree)',
                        'University diploma','University or CNAA first degree (eg BA, B.Sc, B.Ed)','University or CNAA higher degree (eg M.Sc, Ph.D)',
                        'Other technical, professional or higher qualification',"Don't know",'Prefer not to say'], # W6_comb: qeducationW6
                   "Strongly disapprove|Disapprove|Don't know":
                       ["Approve","Disapprove","Don't know"], # approveEUW2 # W7_comb, W10_comb, W13_comb, W8_comb, W9_comb
                   '1 to 24 employees|25 to 499 employees|500 or more employees|':
                       ['1 to 24 employees','25 to 499 employees','500 or more employees',"Don't know"], #fatherNumEmployees,motherNumEmployees #W6_comb,W5_comb,W5_only,W3_comb
                   "Yes, voted|No, did not vote|Don't know":
                       ['Yes',"No","Don't know"],
                   "No, did not vote|Yes, voted|Don't know":
                       ['No','Yes',"Don't know"],
                   "No, did not vote|Yes, voted|2.0":
                       ['No','Yes',"Don't know"],
                   "Strongly disagree|Disagree|Neither nor disagree|Agree|Strongly agree|Don't know":
                       ["Strongly disagree","Disagree","Neither agree nor disagree","Agree","Strongly agree","Don't know"],# euFinancialHelpW2 W3-6_comb
                   "I am very unsure what will happen|I am quite unsure what will happen|I am quite sure what will happen|I am very sure what will happen|Don't know":
                       ["I am very unsure what would happen","I am quite unsure what would happen","I am quite sure what would happen","I am very sure what would happen","Don't know"], # certaintyScotUnionW3 W3-5_comb
                   "0.0|1.0|2.0|3.0|4.0|5.0|6.0|7.0|997.0":
                       ["0 days","1 day","2 days","3 days","4 days","5 days","6 days","7 days","Don't know"], # discussPolDaysW5	W5_comb
                   "A major transfer of powers from Westminster to the Scottish Parliament (\"devo-max\")|Some powers will be transferred but well short of \"devo-max\"|No change to the relationship between Westminster and the Scottish Parliament":
                       ["A major transfer of powers from Westminster to the Scottish Parliament (devo-max)","Some powers will be transferred but well short of devo-max","No change to the relationship between Westminster and the Scottish Parliament"], # expectationManipCheckW1 # W13,10,9,8,7 vs W6-3_comb
                   "No, I did not vote|Yes, I voted|There wasn't a local election in my area|Don't know":
                       ["No, did not vote","Yes, voted","There wasn't a local election in my area","Don't know"], # localTurnoutRetroW2 W3-6_comb
                   "Focuses mainly on criticising other parties|2.0|3.0|4.0|Focuses mainly on putting forward their own policies and personalities|Don't know":
                       ["1 - Focused mainly on criticising other parties","2.0","3.0","4.0","5 - Focused mainly on putting forward their own policies and personalities","Don't know"], # <party>ToneW5 # W5-6_comb, W5_only
                   "Environmental Policy|Defence|Education|Pensions":
                       ["No, I think they *will not* vote","Yes, I think they *will* vote","They are not eligible to vote","Don't know"], # discussantturnoutName1-3W4 # W4-5_comb
                   "Employers in large organisations and higher managerial|Higher professional occupations|Lower professional and managerial and higher supervisory|Intermediate occupations|Employers in small organisations and own account workers|Lower suprivsory and technical occupations|Semi-routine occupations|Routine occupations":
                       ['Employers in large organisations and higher managerial', 'Higher professional occupations',
                        'Lower professional and managerial and higher supervisory', 'Intermediate occupations',
                        'Employers in small organisations and own account workers', 'Lower supervisory and technical occupations',
                        'Semi-routine occupations', 'Routine occupations'], # ns_sec_analytic	 W5_only, W3-6_comb                   
                   "Employers in large organisations and higher managerial|Higher professional occupations|Lower professional and managerail and higher supervisory|Intermediate occupations|Employers in small organisations and own account workers|Lower suprivsory and technical occupations|Semi-routine occupations|Routine occupations":
                       ['Employers in large organisations and higher managerial', 'Higher professional occupations',
                        'Lower professional and managerial and higher supervisory', 'Intermediate occupations',
                        'Employers in small organisations and own account workers', 'Lower supervisory and technical occupations',
                        'Semi-routine occupations', 'Routine occupations'], # ns_sec_analytic	 W5_only, W3-6_comb    # v slight typo!
                   "A major transfer of powers from Westminster to the Scottish Parliament (\"devo-max\")|Some powers will be transferred but well short of \"devo-max\"|No change to the relationship between Westminster and the Scottish Parliament|Don't know":
                       ['A major transfer of powers from Westminster to the Scottish Parliament (devo-max)',
                        'Some powers will be transferred but well short of devo-max',
                        'No change to the relationship between Westminster and the Scottish Parliament',"Don't know"], # expectationManipCheckW1 W3-6_comb
                   "Employers in large establishments|Higher managerial and administrative occupations|L3.1 'Traditional' employees|L3.2 'New' employees|L3.3 'Traditional' self-employed|L3.4 'New' self-employed|L4.1 'Traditional' employees|L4.2 'New' employees|L4.3 'Traditional' self-employed|L4.4 'New' self-employed|Lower managerial and administrative occupations|Higher supervisory occupations|L7.1 Intermediate clerical and administrative occupations|L7.2 Intermediate sales and service occupations|L7.3 Intermediate technical and auxiliary occupations|L7.4 Intermediate engineering occupations|L8.1 Employers in small establishments in industry, commerce, services etc.|L8.2 Employers in small establishments in agriculture|L9.1 Own account workers (non-professional)|L9.2 Own account workers (agriculture)|Lower supervisory occupations|L11.1 Lower technical craft occupations|L11.2 Lower technical process operative occupations|L12.1 Semi-routine sales occupations|L12.2 Semi-routine service occupations|L12.3 Semi-routine technical occupations|L12.4 Semi-routine operative occupations|L12.5 Semi-routine agricultural occupations|L12.6 Semi-routine clerical occupations|L12.7 Semi routine childcare occupations|L13.1 Routine sales and service occupations|L13.2 Routine production occupations|L13.3 Routine technical occupations|L13.4 Routine operative occupations|L13.5 Routine agricultural occupations":
                       ['Employers in large establishments', 'Higher managerial and administrative occupations',
                        'L3.1 Traditional employees', 'L3.2 New employees', 'L3.3 Traditional self-employed',
                        'L3.4 New self-employed', 'L4.1 Traditional employees', 'L4.2 New employees',
                        'L4.3 Traditional self-employed', 'L4.4 New self-employed', 'Lower managerial and administrative occupations',
                        'Higher supervisory occupations', 'L7.1 Intermediate clerical and administrative occupations',
                        'L7.2 Intermediate sales and service occupations', 'L7.3 Intermediate technical and auxiliary occupations',
                        'L7.4 Intermediate engineering occupations', 'L8.1 Employers in small establishments in industry, commerce, services etc.',
                        'L8.2 Employers in small establishments in agriculture', 'L9.1 Own account workers (non-professional)',
                        'L9.2 Own account workers (agriculture)', 'Lower supervisory occupations', 'L11.1 Lower technical craft occupations',
                        'L11.2 Lower technical process operative occupations', 'L12.1 Semi-routine sales occupations',
                        'L12.2 Semi-routine service occupations', 'L12.3 Semi-routine technical occupations', 'L12.4 Semi-routine operative occupations',
                        'L12.5 Semi-routine agricultural occupations', 'L12.6 Semi-routine clerical occupations', 'L12.7 Semi routine childcare occupations',
                        'L13.1 Routine sales and service occupations', 'L13.2 Routine production occupations', 'L13.3 Routine technical occupations',
                        'L13.4 Routine operative occupations', 'L13.5 Routine agricultural occupations'],
                   "Employers in large establishments|Higher managerial and administrative occupations|L3.1 ?Traditional? employees|L3.2 ?New? employees|L3.3 ?Traditional? self-employed|L3.4 ?New? self-employed|L4.1 ?Traditional? employees|L4.2 ?New? employees|L4.3 ?Traditional? self-employed|L4.4 ?New? self-employed|Lower managerial and administrative occupations|Higher supervisory occupations|L7.1 Intermediate clerical and administrative occupations|L7.2 Intermediate sales and service occupations|L7.3 Intermediate technical and auxiliary occupations|L7.4 Intermediate engineering occupations|L8.1 Employers in small establishments in industry, commerce, services etc.|L8.2 Employers in small establishments in agriculture|L9.1 Own account workers (non-professional)|L9.2 Own account workers (agriculture)|Lower supervisory occupations|L11.1 Lower technical craft occupations|L11.2 Lower technical process operative occupations|L12.1 Semi-routine sales occupations|L12.2 Semi-routine service occupations|L12.3 Semi-routine technical occupations|L12.4 Semi-routine operative occupations|L12.5 Semi-routine agricultural occupations|L12.6 Semi-routine clerical occupations|L12.7 Semi routine childcare occupations|L13.1 Routine sales and service occupations|L13.2 Routine production occupations|L13.3 Routine technical occupations|L13.4 Routine operative occupations|L13.5 Routine agricultural occupations":
                       ['Employers in large establishments', 'Higher managerial and administrative occupations',
                        'L3.1 Traditional employees', 'L3.2 New employees', 'L3.3 Traditional self-employed',
                        'L3.4 New self-employed', 'L4.1 Traditional employees', 'L4.2 New employees',
                        'L4.3 Traditional self-employed', 'L4.4 New self-employed', 'Lower managerial and administrative occupations',
                        'Higher supervisory occupations', 'L7.1 Intermediate clerical and administrative occupations',
                        'L7.2 Intermediate sales and service occupations', 'L7.3 Intermediate technical and auxiliary occupations',
                        'L7.4 Intermediate engineering occupations', 'L8.1 Employers in small establishments in industry, commerce, services etc.',
                        'L8.2 Employers in small establishments in agriculture', 'L9.1 Own account workers (non-professional)',
                        'L9.2 Own account workers (agriculture)', 'Lower supervisory occupations', 'L11.1 Lower technical craft occupations',
                        'L11.2 Lower technical process operative occupations', 'L12.1 Semi-routine sales occupations',
                        'L12.2 Semi-routine service occupations', 'L12.3 Semi-routine technical occupations', 'L12.4 Semi-routine operative occupations',
                        'L12.5 Semi-routine agricultural occupations', 'L12.6 Semi-routine clerical occupations', 'L12.7 Semi routine childcare occupations',
                        'L13.1 Routine sales and service occupations', 'L13.2 Routine production occupations', 'L13.3 Routine technical occupations',
                        'L13.4 Routine operative occupations', 'L13.5 Routine agricultural occupations'],
                   "1|2":
                       ["No","Yes"], # tryReduceImmigDKW4, achieveReduceImmigUKIPW4, achieveReduceImmigGrnW4, achieveReduceImmigDKW4, tryReduceInequalityDKW4, successReduceInequalityDKW4 # W4-5_comb # sharedContentOnline_1-5W4 W5_comb # voteMethodEurope_dkW2, discussantsAskedYouToVote_DKW2 ,discussantsAccompaniedVote_dkW2, referendumContact_dkW2 # W3_comb
                   "1.0|2.0":
                       ["No","Yes"], # tryReduceImmigDKW4, achieveReduceImmigUKIPW4, achieveReduceImmigGrnW4, achieveReduceImmigDKW4, tryReduceInequalityDKW4, successReduceInequalityDKW4 # W4-5_comb # sharedContentOnline_1-5W4 W5_comb # voteMethodEurope_dkW2, discussantsAskedYouToVote_DKW2 ,discussantsAccompaniedVote_dkW2, referendumContact_dkW2 # W3_comb
                   "Should definitely be illegal|Should probably be illegal|Should probably be legal|Should definitely be legal|5.0":
                       ["Should definitely be illegal","Should probably be illegal","Should probably be legal","Should definitely be legal","Don't know"], # zeroHourContractW6
                  }


rename_variable_dict = pd.DataFrame.from_dict( {k : "|".join(v) for k, v in rename_cat_dict.items()} , orient='index' ).reset_index()
rename_variable_dict.columns = ["original_cat_list","renameed_cat_list"]
rename_variable_dict.to_csv( BES_small_data_files + "rename_variable_dict.csv" )

def re_name(ques):
    if ques in rename_cat_dict.keys():
        return "|".join( rename_cat_dict[ques] )
    else:
        return ques

In [361]:
col = "scotReferendumIntentionW6"
BES_Panel[ col ].value_counts()
# full_set = BES_Panel[ col ].cat.categories
# full_set

Scotland should become an independent country    2490
111.0                                            2424
99.0                                              357
2.0                                                42
Name: scotReferendumIntentionW6, dtype: int64

In [362]:
BES_Panel[ col ].cat.categories

Index(['Scotland should become an independent country', 2.0, 99.0, 111.0], dtype='object')

In [351]:
No, I think they *will not* vote|Yes, I think they *will* vote|Don't know

0                                   NaN
1                                   NaN
2                                   NaN
3                                   NaN
4                                   NaN
5                                   NaN
6                                   NaN
7                                   NaN
8                                   NaN
9                                   NaN
10                                  NaN
11                                  NaN
12                                  NaN
13                                  NaN
14                                  NaN
15                                  NaN
16                                  NaN
17                                  NaN
18                                  NaN
19                                  NaN
20                                  NaN
21                                  NaN
22                                  NaN
23                                  NaN
24                                  NaN


In [348]:
# {k:v in zip(half_set,full_set)}

dict(zip(half_set,full_set))

{'No, I do not regard myself as belonging to any particular religion.': 'No, I do not regard myself as belonging to any particular religion.',
 'Yes - Church of England/Anglican/Episcopal': 'Yes - Church of England/Anglican/Episcopal',
 'Yes - Roman Catholic': 'Yes - Roman Catholic',
 'Yes - Presbyterian/Church of Scotland': 'Yes - Presbyterian/Church of Scotland',
 'Yes - Methodist': 'Yes - Methodist',
 'Yes - Baptist': 'Yes - Baptist',
 'Yes - United Reformed Church': 'Yes - United Reformed Church',
 'Yes - Free Presbyterian': 'Yes - Free Presbyterian',
 'Yes - Brethren': 'Yes - Brethren',
 'Yes - Judaism': 'Yes - Judaism',
 'Yes - Hinduism': 'Yes - Hinduism',
 'Yes - Islam': 'Yes - Islam',
 'Yes - Sikhism': 'Yes - Sikhism',
 'Yes - Buddhism': 'Yes - Buddhism',
 'Yes - Other': 'Yes - Other',
 16.0: 'Prefer not to say',
 17.0: 'Yes - Orthodox Christian',
 18.0: 'Yes - Pentecostal',
 19.0: 'Yes - Evangelical /independent/non-denominational'}

In [330]:
col = "profile_religion"
BES_Panel[ col ].value_counts()

No, I do not regard myself as belonging to any particular religion.                                                         14859
Yes - Church of England/Anglican/Episcopal                                                                                   8285
Yes - Roman Catholic                                                                                                         2329
Yes - Presbyterian/Church of Scotland                                                                                        1022
Prefer not to say                                                                                                             788
Yes - Methodist                                                                                                               744
Yes - Other                                                                                                                   737
Yes - Baptist                                                                             

In [308]:
join_list = "|".join( BES_Panel[ col ].cat.categories )
join_list

"1 to 24 employees|25 to 499 employees|Don't know"

In [304]:
fix_a_hat_chars(join_list)

['No, I do not regard myself as belonging to any particular religion.',
 'Yes - Church of England/Anglican/Episcopal',
 'Yes - Roman Catholic',
 'Yes - Presbyterian/Church of Scotland',
 'Yes - Methodist',
 'Yes - Baptist',
 'Yes - United Reformed Church',
 'Yes - Free Presbyterian',
 'Yes - Brethren',
 'Yes - Judaism',
 'Yes - Hinduism',
 'Yes - Islam',
 'Yes - Sikhism',
 'Yes - Buddhism',
 'Yes - Other',
 'Prefer not to say',
 'Yes - Orthodox Christian',
 'Yes - Pentecostal (e.g. Assemblies of God, Elim Pentecostal Church, New Testament Church of God, Redeemed Christian Chur',
 'Yes - Evangelical - independent/non-denominational (e.g. FIEC, Pioneer, Vineyard, Newfrontiers)']

In [305]:
"|".join( fix_a_hat_chars(join_list) )

'No, I do not regard myself as belonging to any particular religion.|Yes - Church of England/Anglican/Episcopal|Yes - Roman Catholic|Yes - Presbyterian/Church of Scotland|Yes - Methodist|Yes - Baptist|Yes - United Reformed Church|Yes - Free Presbyterian|Yes - Brethren|Yes - Judaism|Yes - Hinduism|Yes - Islam|Yes - Sikhism|Yes - Buddhism|Yes - Other|Prefer not to say|Yes - Orthodox Christian|Yes - Pentecostal (e.g. Assemblies of God, Elim Pentecostal Church, New Testament Church of God, Redeemed Christian Chur|Yes - Evangelical - independent/non-denominational (e.g. FIEC, Pioneer, Vineyard, Newfrontiers)'

In [255]:
def fix_100_seq(col, start, finish, weasel, rng=100):
    lst = list([weasel,start, finish])
    lst_dict = {"0":start,str(rng):finish}

    fullseq = [start]
    [fullseq.append(str(x)) for x in range(1,rng)]
    fullseq.append(finish)
    fullseq.append(weasel)
    # make sure all numbers in same format (string integers)
    BES_Panel[col] = BES_Panel[col].cat.rename_categories( [str(int(x)) if x not in lst else x for x in BES_Panel[col].cat.categories ] )
    BES_Panel[col] = BES_Panel[col].cat.rename_categories( [lst_dict[x] if x in lst_dict.keys() else x for x in BES_Panel[col].cat.categories ] )
    
    # change categories to correct range
    BES_Panel[col] = BES_Panel[col].cat.set_categories(fullseq)
    if len( BES_Panel[col].cat.categories ) != rng+2:
        raise Exception("wrong number of categories!")

In [272]:
# start = "Allow many fewer"
# finish = "Allow many more"
# weasel = "Don't know"        
# fix_100_seq(col, start, finish, weasel,10)    

In [299]:
a

'No, I do not regard myself as belonging to any particular religion.|Yes - Church of England/Anglican/Episcopal|Yes - Roman Catholic|Yes - Presbyterian/Church of Scotland|Yes - Methodist|Yes - Baptist|Yes - United Reformed Church|Yes - Free Presbyterian|Yes - Brethren|Yes - Judaism|Yes - Hinduism|Yes - Islam|Yes - Sikhism|Yes - Buddhism|Yes - Other|Prefer not to say|Yes Orthodox Christian|Yes - Pentecostal (e.g. Assemblies of God, Elim Pentecostal Church, New Testament Church of God, Redeemed Christian Chur|Yes - Evangelical independent/non-denominational (e.g. FIEC, Pioneer, Vineyard, Newfrontiers)'

In [300]:
b

'No, I do not regard myself as belonging to any particular religion.|Yes - Church of England/Anglican/Episcopal|Yes - Roman Catholic|Yes - Presbyterian/Church of Scotland|Yes - Methodist|Yes - Baptist|Yes - United Reformed Church|Yes - Free Presbyterian|Yes - Brethren|Yes - Judaism|Yes - Hinduism|Yes - Islam|Yes - Sikhism|Yes - Buddhism|Yes - Other|Prefer not to say|Yes - Orthodox Christian|Yes - Pentecostal (e.g. Assemblies of God, Elim Pentecostal Church, New Testament Church of God, Redeemed Christian Chur|Yes - Evangelical - independent/non-denominational (e.g. FIEC, Pioneer, Vineyard, Newfrontiers)'

In [325]:
test = "No, I do not regard myself as belonging to any particular religion.|Yes - Church of England/Anglican/Episcopal|Yes - Roman Catholic|Yes - Presbyterian/Church of Scotland|Yes - Methodist|Yes - Baptist|Yes - United Reformed Church|Yes - Free Presbyterian|Yes - Brethren|Yes - Judaism|Yes - Hinduism|Yes - Islam|Yes - Sikhism|Yes - Buddhism|Yes - Other|Prefer not to say|Yes – Orthodox Christian|Yes - Pentecostal (e.g. Assemblies of God, Elim Pentecostal Church, New Testament Church of God, Redeemed Christian Chur|Yes - Evangelical – independent/non-denominational (e.g. FIEC, Pioneer, Vineyard, Newfrontiers)"

In [327]:
test2 = "No, I do not regard myself as belonging to any particular religion.|Yes - Church of England/Anglican/Episcopal|Yes - Roman Catholic|Yes - Presbyterian/Church of Scotland|Yes - Methodist|Yes - Baptist|Yes - United Reformed Church|Yes - Free Presbyterian|Yes - Brethren|Yes - Judaism|Yes - Hinduism|Yes - Islam|Yes - Sikhism|Yes - Buddhism|Yes - Other|16.0|17.0|18.0|19.0"

In [328]:
len( test2.split("|") )

19

In [324]:
{el:el for el in ns_sec.split("|")}

{'Employers in large establishments': 'Employers in large establishments',
 'Higher managerial and administrative occupations': 'Higher managerial and administrative occupations',
 'Higher supervisory occupations': 'Higher supervisory occupations',
 'L11.1 Lower technical craft occupations': 'L11.1 Lower technical craft occupations',
 'L11.2 Lower technical process operative occupations': 'L11.2 Lower technical process operative occupations',
 'L12.1 Semi-routine sales occupations': 'L12.1 Semi-routine sales occupations',
 'L12.2 Semi-routine service occupations': 'L12.2 Semi-routine service occupations',
 'L12.3 Semi-routine technical occupations': 'L12.3 Semi-routine technical occupations',
 'L12.4 Semi-routine operative occupations': 'L12.4 Semi-routine operative occupations',
 'L12.5 Semi-routine agricultural occupations': 'L12.5 Semi-routine agricultural occupations',
 'L12.6 Semi-routine clerical occupations': 'L12.6 Semi-routine clerical occupations',
 'L12.7 Semi routine childc

In [317]:
ns_sec = "Employers in large establishments|Higher managerial and administrative occupations|L3.1 Traditional employees|L3.2 New employees|L3.3 Traditional self-employed|L3.4 New self-employed|L4.1 Traditional employees|L4.2 New employees|L4.3 Traditional self-employed|L4.4 New self-employed|Lower managerial and administrative occupations|Higher supervisory occupations|L7.1 Intermediate clerical and administrative occupations|L7.2 Intermediate sales and service occupations|L7.3 Intermediate technical and auxiliary occupations|L7.4 Intermediate engineering occupations|L8.1 Employers in small establishments in industry, commerce, services etc.|L8.2 Employers in small establishments in agriculture|L9.1 Own account workers (non-professional)|L9.2 Own account workers (agriculture)|Lower supervisory occupations|L11.1 Lower technical craft occupations|L11.2 Lower technical process operative occupations|L12.1 Semi-routine sales occupations|L12.2 Semi-routine service occupations|L12.3 Semi-routine technical occupations|L12.4 Semi-routine operative occupations|L12.5 Semi-routine agricultural occupations|L12.6 Semi-routine clerical occupations|L12.7 Semi routine childcare occupations|L13.1 Routine sales and service occupations|L13.2 Routine production occupations|L13.3 Routine technical occupations|L13.4 Routine operative occupations|L13.5 Routine agricultural occupations"

In [320]:
dict(ns_sec,ns_sec)

TypeError: dict expected at most 1 arguments, got 2

In [298]:
import difflib

cases=[(a, b)
       ] 

for a,b in cases:     
    print('{} => {}'.format(a,b))  
    for i,s in enumerate(difflib.ndiff(a, b)):
        if s[0]==' ': continue
        elif s[0]=='-':
            print(u'Delete "{}" from position {}'.format(s[-1],i))
        elif s[0]=='+':
            print(u'Add "{}" to position {}'.format(s[-1],i))    
    print()      

No, I do not regard myself as belonging to any particular religion.|Yes - Church of England/Anglican/Episcopal|Yes - Roman Catholic|Yes - Presbyterian/Church of Scotland|Yes - Methodist|Yes - Baptist|Yes - United Reformed Church|Yes - Free Presbyterian|Yes - Brethren|Yes - Judaism|Yes - Hinduism|Yes - Islam|Yes - Sikhism|Yes - Buddhism|Yes - Other|Prefer not to say|Yes Orthodox Christian|Yes - Pentecostal (e.g. Assemblies of God, Elim Pentecostal Church, New Testament Church of God, Redeemed Christian Chur|Yes - Evangelical independent/non-denominational (e.g. FIEC, Pioneer, Vineyard, Newfrontiers) => No, I do not regard myself as belonging to any particular religion.|Yes - Church of England/Anglican/Episcopal|Yes - Roman Catholic|Yes - Presbyterian/Church of Scotland|Yes - Methodist|Yes - Baptist|Yes - United Reformed Church|Yes - Free Presbyterian|Yes - Brethren|Yes - Judaism|Yes - Hinduism|Yes - Islam|Yes - Sikhism|Yes - Buddhism|Yes - Other|Prefer not to say|Yes - Orthodox Christia

In [296]:
c = return_not_ascii(b)
c

''

In [284]:
def return_not_ascii(s): # ascii or £
    if not isinstance(s,str):
        return None
    return "|".join([ c for c in s if not ((ord(c) < 128) or (ord(c)==163)) ])

In [271]:
def fix_a_hat_chars(cat_string):
    cat_array = cat_string.split("|")
    a_hat_present = False
    for el_no in range( 0, len(cat_array) ):
        el = cat_array[el_no]
        el = re.sub( "SiÃƒÂ¢n C. Jame|SiÃ¢n C. James|SiÃ¢n C. Jame|Siân C. James", "Sian C. James", el)
        el = re.sub( "ThÃ©rÃ¨se  Coff|Thérèse  Coffey", "Therese  Coffey", el)
        el = re.sub( "RA©union|RÃ©union|RAÂ©union|RÃƒÂ©union", "Reunion", el)
        el = re.sub( "\xa0Lower supervisory occupations", "Lower supervisory occupations", el)
        el = re.sub( "Don‘t know|Don?t know|Dona??t know|Dona€™t know|Donâ€™t know|Don’t know|Don‘t know|Don\x91t know|Don\x92t know|Dona\x80\x99t know|Do\x92t know","Don't know", el  )
        el = re.sub( "Â–|\x96|–", "-", el )
        el = re.sub( "Â£|\xc2£", "£", el )
        el = re.sub( "\xa0|\sâ€“\s|\s\xe2\x80\x93\s|\sÂ‘|Â’\s" , " ", el )
        el = re.sub( "Â‘|Â’|Â€Â™|â\?\?|\x80\x99|â€™|\xe2\x80\x99|â|â\x80\x99|\?\?|\x92|‘|\x91|’", "'", el )
        
        if el != cat_array[el_no]:
            a_hat_present = True
            cat_array[el_no] = el
            
    if a_hat_present:
        return cat_array
    else:
        return None

In [142]:
"|".join( fix_a_hat_chars(test) )

"Employers in large establishments|Higher managerial and administrative occupations|L3.1 'Traditional' employees|L3.2 'New' employees|L3.3 'Traditional' self-employed|L3.4 'New' self-employed|L4.1 'Traditional' employees|L4.2 'New' employees|L4.3 'Traditional' self-employed|L4.4 'New' self-employed|Lower managerial and administrative occupations|Higher supervisory occupations|L7.1 Intermediate clerical and administrative occupations|L7.2 Intermediate sales and service occupations|L7.3 Intermediate technical and auxiliary occupations|L7.4 Intermediate engineering occupations|L8.1 Employers in small establishments in industry, commerce, services etc.|L8.2 Employers in small establishments in agriculture|L9.1 Own account workers (non-professional)|L9.2 Own account workers (agriculture)|Lower supervisory occupations|L11.1 Lower technical craft occupations|L11.2 Lower technical process operative occupations|L12.1 Semi-routine sales occupations|L12.2 Semi-routine service occupations|L12.3 Se

In [143]:
fix_a_hat_chars(test)


['Employers in large establishments',
 'Higher managerial and administrative occupations',
 "L3.1 'Traditional' employees",
 "L3.2 'New' employees",
 "L3.3 'Traditional' self-employed",
 "L3.4 'New' self-employed",
 "L4.1 'Traditional' employees",
 "L4.2 'New' employees",
 "L4.3 'Traditional' self-employed",
 "L4.4 'New' self-employed",
 'Lower managerial and administrative occupations',
 'Higher supervisory occupations',
 'L7.1 Intermediate clerical and administrative occupations',
 'L7.2 Intermediate sales and service occupations',
 'L7.3 Intermediate technical and auxiliary occupations',
 'L7.4 Intermediate engineering occupations',
 'L8.1 Employers in small establishments in industry, commerce, services etc.',
 'L8.2 Employers in small establishments in agriculture',
 'L9.1 Own account workers (non-professional)',
 'L9.2 Own account workers (agriculture)',
 'Lower supervisory occupations',
 'L11.1 Lower technical craft occupations',
 'L11.2 Lower technical process operative occupat

In [144]:
# s.cat.rename_categories([1,2,3])
# EUContactRemainConW8|EUContactRemainLabW8|EUContactRemainLDW8|
# EUContactRemainSNPW8|EUContactRemainPCW8|EUContactRemainUKIPW8|
# EUContactRemainGreenW8|EUContactRemainOthW8|EUContactRemainNoneW8|
# EUContactRemainDKW8|EUContactLeaveConW8|EUContactLeaveLabW8|
# EUContactLeaveLDW8|EUContactLeaveSNPW8|EUContactLeavePCW8|
# EUContactLeaveUKIPW8|EUContactLeaveGreenW8|EUContactLeaveOthW8|
# EUContactLeaveNoneW8|EUContactLeaveDKW8

# pattern match "EUContact*****W8"
# debateOneWatchW8|debateTwoWatchW8

# "1.0|2.0|99.0" -> 

# euRefVoteSqueezeW7 "Will not vote|Yes - Leave|Yes - Remain|No - not decided"
#    -> Stay/remain in the EU|Leave the EU|I would/will not vote|Don't know
#    HMM - RENAME AND REORDER!

# miieuW7
# "Issue stated|Nothing|Don't know" -> "Issue stated|None|Don't know"
# MIIEUW8
# "1.0|Nothing|Don't know" -> "Issue stated|None|Don't know"
# partyIdEUW7|partyIdEUW8
# "Mainly leave|Mainly remain|Fairly evenly split|Don't know" -> "Mainly remain|Fairly evenly divided|Mainly leave|Don't know"
#    HMM - RENAME AND REORDER!

# 1. campaignVisionYesW3|campaignVisionNoW3, govtNatSecuritySuccessW4
# Very unsuccessful|Fairly unsuccessful|Neither successful nor unsuccessful|Fairly successful|Very successful|Don't know
# Very unsuccessful|Somewhat unsuccessful|Neither successful or unsuccessful|Somewhat successful|Very successful|Don't know

# Fairly <-> Somewhat

# 2. euroTurnoutW1, scotReferendumTurnoutW1|scotReferendumTurnoutW2|welshTurnoutW7|scotTurnoutW7, turnoutUKGeneralW1|turnoutUKGeneralW2|turnoutUKGeneralW3|turnoutUKGeneralW4|turnoutUKGeneralW5|euRefTurnoutW7|euRefTurnoutW8
# Very unlikely that I vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I vote|Don't know
# Very unlikely that I would vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I would vote|Don't know
# There are not local elections in my area
    #|Very unlikely that I will vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I will vote|Don't know
# Very unlikely that I will vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I will vote|Don't know

# "Very unlikely that I vote", "Very unlikely that I would vote" ->  "Very unlikely that I will vote" 

rename_cat_dict = {"North East|North West": [ "No", "Yes" ],
                   "1.0|2.0|99.0": ["No", "Yes", "99.0"],
                   "Will not vote|Yes - Leave|Yes - Remain|No - not decided":
                       ['I would/will not vote', 'Leave the EU','Stay in the EU', "Don't know"], 
                   "Stay/remain in the EU|Leave the EU|I would/will not vote|Don't know":
                       ['Stay in the EU','Leave the EU',  'I would/will not vote', "Don't know"],   # euRefVote    
                   "Stay/remain in the EU|Leave the EU|Don't know":
                       ['Stay in the EU','Leave the EU', "Don't know"],   # profile_eurefvote                    
                   "Issue stated|Nothing|Don't know":  ['Issue stated', 'None', "Don't know"],
                   "1.0|Nothing|Don't know":           ['Issue stated', 'None', "Don't know"],
                   "a|b|C1|C2|d|e|Refused|Unknown" : ['A', 'B', 'C1', 'C2', 'D', 'E', 'Refused', 'Unknown'],
                   "Mainly leave|Mainly remain|Fairly evenly split|Don't know":
                       ['Mainly leave','Mainly remain', 'Fairly evenly divided', "Don't know"],
                   "Very unsuccessful|Somewhat unsuccessful|Neither successful or unsuccessful|Somewhat successful|Very successful|Don't know":
                       ['Very unsuccessful', 'Fairly unsuccessful', 'Neither successful nor unsuccessful',
                        'Fairly successful', 'Very successful', "Don't know"],
                   "Very unlikely that I vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I vote|Don't know":
                       ['Very unlikely that I will vote', 'Fairly unlikely', 'Neither likely nor unlikely',
                        'Fairly likely', 'Very likely that I will vote', "Don't know"],
                   "Very unlikely that I would vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I would vote|Don't know":
                       ['Very unlikely that I will vote', 'Fairly unlikely', 'Neither likely nor unlikely',
                        'Fairly likely', 'Very likely that I will vote', "Don't know"],
                   "No, did not vote|Yes, voted|3.0|4.0|5.0|Don't know":   
                       ["Very unlikely that I will vote", "Fairly unlikely", 'Neither likely nor unlikely',
                        "Fairly likely", "Very likely that I will vote", "Don't know"], #londonTurnoutW7
                   'No, I do not regard myself as belonging to any particular religion.|Yes - Church of England/Anglican/Episcopal|Yes - Roman Catholic|Yes - Presbyterian/Church of Scotland|Yes - Methodist|Yes - Baptist|Yes - United Reformed Church|Yes - Free Presbyterian|Yes - Brethren|Yes - Judaism|Yes - Hinduism|Yes - Islam|Yes - Sikhism|Yes - Buddhism|Yes - Other|Prefer not to say|Yes Orthodox Christian|Yes - Pentecostal (e.g. Assemblies of God, Elim Pentecostal Church, New Testament Church of God, Redeemed Christian Chur|Yes - Evangelical independent/non-denominational (e.g. FIEC, Pioneer, Vineyard, Newfrontiers)':
                       ["No, I do not regard myself as belonging to any particular religion.","Yes - Church of England/Anglican/Episcopal",
                        "Yes - Roman Catholic","Yes - Presbyterian/Church of Scotland","Yes - Methodist","Yes - Baptist",
                        "Yes - United Reformed Church","Yes - Free Presbyterian","Yes - Brethren","Yes - Judaism","Yes - Hinduism",
                        "Yes - Islam","Yes - Sikhism","Yes - Buddhism","Yes - Other","Prefer not to say","Yes - Orthodox Christian",
                        "Yes - Pentecostal","Yes - Evangelical /independent/non-denominational"], #xprofile_religionW10
                   'Own - outright|Own - with a mortgage|Own (part-own) - through shared ownership scheme (i.e. pay part mortgage, part rent)|Rent - from a private landlord|Rent - from my local authority|Rent - from a housing association|Neither - I live with my parents, family or friends but pay some rent to them|Neither - I live rent-free with my parents, family or friends|Other':
                       [ 'Own - outright',
                         'Own - with a mortgage',
                         'Own (part-own) - through shared ownership scheme (i.e. pay part mortgage, part rent)',
                         'Rent - from a private landlord',
                         'Rent - from my local authority',
                         'Rent - from a housing association',
                         'Neither - I live with my parents, family or friends but pay some rent to them',
                         'Neither - I live rent-free with my parents, family or friends',
                         'Other'], #profile_house_tenureW11|profile_house_tenureW12|profile_house_tenureW13
                   "I voted 'No' (Scotland should not be an independent country)|I voted 'Yes' (Scotland should be an independent country)|111.0|Don't know":
                       ["No","Yes","Did not vote","Don't know"], # referendumrecall
                   "Voted Yes|Voted No|Did not vote|Can't remember":
                       ["Yes","No","Did not vote","Don't know"], # scotRefVoteW4_
                   "No|Yes|3.0|Don't know":
                       ["No","Yes","Did not vote","Don't know"], # regretsIHaveAFewEUW10|regretsIHaveAFewEUW11   
                   "No|Yes|3|Don't know":
                       ["No","Yes","Did not vote","Don't know"], # regretsIHaveAFewEU W11_only|regretsIHaveAFew W10_only 
                   "Professional or higher technical work - work that requires at least degree-level qualifications (e.g. doctor, accountant|Manager or Senior Administrator (e.g. company director, finance manager, personnel manager, senior sales manager, senior|Clerical (e.g. clerk, secretary)|Sales or Services (e.g. commercial traveller, shop assistant, nursery nurse, care assistant, paramedic)|Foreman or Supervisor of Other Workers (e.g building site foreman, supervisor of cleaning workers)|Skilled Manual Work (e.g. plumber, electrician, fitter)|Semi-Skilled or Unskilled Manual Work (e.g. machine operator, assembler, postman, waitress, cleaner, labourer, driver, b|Other|Have never worked":
                       ['Professional or higher technical work / higher managerial - work that requires at least degree-level qualifications (e.g',
                        'Manager or Senior Administrator / intermediate managerial / professional (e.g. company director, finance manager, person',
                        'Clerical/junior managerial/professional/administrator (e.g. office worker, student doctor, sales person, clerk, secretar',
                        'Sales or Services (e.g. commercial traveller, shop assistant, nursery nurse, care assistant, paramedic)',
                        'Foreman or Supervisor of Other Workers (e.g. building site foreman, supervisor of cleaning workers)',
                        'Skilled Manual Work (e.g. plumber, electrician, fitter)',
                        'Semi-Skilled or Unskilled Manual Work (e.g. machine operator, assembler, postman, waitress, cleaner, labourer, driver, b',
                        'Other',
                        'Have never worked'], # work_type -> profile_work_typeW7
                   "No formal qualifications|Youth training certificate/skillseekers|Recognised trade apprenticeship completed|Clerical and commercial|City & Guilds certificate|City & Guilds certificate - advanced|onc|CSE grades 2-5|CSE grade 1, GCE O level, GCSE, School Certificate|Scottish Ordinary/ Lower Certificate|GCE A level or Higher Certificate|Scottish Higher Certificate|Nursing qualification (eg SEN, SRN, SCM, RGN)|Teaching qualification (not degree)|University diploma|University or CNAA first degree (eg BA, B.Sc, B.Ed)|University or CNAA higher degree (eg M.Sc, Ph.D)|Other technical, professional or higher qualification|Don't know|Prefer not to say":
                       ['No formal qualifications','Youth training certificate/skillseekers','Recognised trade apprenticeship completed',
                        'Clerical and commercial','City and Guild certificate','City and Guild certificate - advanced','onc','CSE grades 2-5',
                        'CSE grade 1, GCE O level, GCSE, School Certificate','Scottish Ordinary/ Lower Certificate','GCE A level or Higher Certificate',
                        'Scottish Higher Certificate','Nursing qualification (eg SEN, SRN, SCM, RGN)','Teaching qualification (not degree)',
                        'University diploma','University or CNAA first degree (eg BA, B.Sc, B.Ed)','University or CNAA higher degree (eg M.Sc, Ph.D)',
                        'Other technical, professional or higher qualification',"Don't know",'Prefer not to say'], # W6_comb: qeducationW6
                   "Strongly disapprove|Disapprove|Don't know":
                       ["Approve","Disapprove","Don't know"], # approveEUW2 # W7_comb, W10_comb, W13_comb, W8_comb, W9_comb
                   '1 to 24 employees|25 to 499 employees|500 or more employees|':
                       ['1 to 24 employees','25 to 499 employees','500 or more employees',"Don't know"], #fatherNumEmployees,motherNumEmployees #W6_comb,W5_comb,W5_only,W3_comb
                   "No, did not vote|Yes, voted|Don't know":
                       ['No','Yes',"Don't know"],
                   "Strongly disagree|Disagree|Neither nor disagree|Agree|Strongly agree|Don't know":
                       ["Strongly disagree","Disagree","Neither agree nor disagree","Agree","Strongly agree","Don't know"],# euFinancialHelpW2 W3-6_comb
                   "I am very unsure what will happen|I am quite unsure what will happen|I am quite sure what will happen|I am very sure what will happen|Don't know":
                       ["I am very unsure what would happen","I am quite unsure what would happen","I am quite sure what would happen","I am very sure what would happen","Don't know"], # certaintyScotUnionW3 W3-5_comb
                   "0.0|1.0|2.0|3.0|4.0|5.0|6.0|7.0|997.0":
                       ["0 days","1 day","2 days","3 days","4 days","5 days","6 days","7 days","Don't know"], # discussPolDaysW5	W5_comb
                   "A major transfer of powers from Westminster to the Scottish Parliament (\"devo-max\")|Some powers will be transferred but well short of \"devo-max\"|No change to the relationship between Westminster and the Scottish Parliament":
                       ["A major transfer of powers from Westminster to the Scottish Parliament (devo-max)","Some powers will be transferred but well short of devo-max","No change to the relationship between Westminster and the Scottish Parliament"], # expectationManipCheckW1 # W13,10,9,8,7 vs W6-3_comb
                   "No, I did not vote|Yes, I voted|There wasn't a local election in my area|Don't know":
                       ["No, did not vote","Yes, voted","There wasn't a local election in my area","Don't know"], # localTurnoutRetroW2 W3-6_comb
                   "Focuses mainly on criticising other parties|2.0|3.0|4.0|Focuses mainly on putting forward their own policies and personalities|Don't know":
                       ["1 - Focused mainly on criticising other parties","2.0","3.0","4.0","5 - Focused mainly on putting forward their own policies and personalities","Don't know"], # <party>ToneW5 # W5-6_comb, W5_only
                   "Environmental Policy|Defence|Education|Pensions":
                       ["No, I think they *will not* vote","Yes, I think they *will* vote","They are not eligible to vote","Don't know"], # discussantturnoutName1-3W4 # W4-5_comb
                   "Employers in large organisations and higher managerial|Higher professional occupations|Lower professional and managerial and higher supervisory|Intermediate occupations|Employers in small organisations and own account workers|Lower suprivsory and technical occupations|Semi-routine occupations|Routine occupations":
                       ['Employers in large organisations and higher managerial', 'Higher professional occupations',
                        'Lower professional and managerial and higher supervisory', 'Intermediate occupations',
                        'Employers in small organisations and own account workers', 'Lower supervisory and technical occupations',
                        'Semi-routine occupations', 'Routine occupations'], # ns_sec_analytic	 W5_only, W3-6_comb                   
                   "Employers in large organisations and higher managerial|Higher professional occupations|Lower professional and managerail and higher supervisory|Intermediate occupations|Employers in small organisations and own account workers|Lower suprivsory and technical occupations|Semi-routine occupations|Routine occupations":
                       ['Employers in large organisations and higher managerial', 'Higher professional occupations',
                        'Lower professional and managerial and higher supervisory', 'Intermediate occupations',
                        'Employers in small organisations and own account workers', 'Lower supervisory and technical occupations',
                        'Semi-routine occupations', 'Routine occupations'], # ns_sec_analytic	 W5_only, W3-6_comb    # v slight typo!
                   "A major transfer of powers from Westminster to the Scottish Parliament (\"devo-max\")|Some powers will be transferred but well short of \"devo-max\"|No change to the relationship between Westminster and the Scottish Parliament|Don't know":
                       ['A major transfer of powers from Westminster to the Scottish Parliament (devo-max)',
                        'Some powers will be transferred but well short of devo-max',
                        'No change to the relationship between Westminster and the Scottish Parliament',"Don't know"], # expectationManipCheckW1 W3-6_comb
                   "Employers in large establishments|Higher managerial and administrative occupations|L3.1 'Traditional' employees|L3.2 'New' employees|L3.3 'Traditional' self-employed|L3.4 'New' self-employed|L4.1 'Traditional' employees|L4.2 'New' employees|L4.3 'Traditional' self-employed|L4.4 'New' self-employed|Lower managerial and administrative occupations|Higher supervisory occupations|L7.1 Intermediate clerical and administrative occupations|L7.2 Intermediate sales and service occupations|L7.3 Intermediate technical and auxiliary occupations|L7.4 Intermediate engineering occupations|L8.1 Employers in small establishments in industry, commerce, services etc.|L8.2 Employers in small establishments in agriculture|L9.1 Own account workers (non-professional)|L9.2 Own account workers (agriculture)|Lower supervisory occupations|L11.1 Lower technical craft occupations|L11.2 Lower technical process operative occupations|L12.1 Semi-routine sales occupations|L12.2 Semi-routine service occupations|L12.3 Semi-routine technical occupations|L12.4 Semi-routine operative occupations|L12.5 Semi-routine agricultural occupations|L12.6 Semi-routine clerical occupations|L12.7 Semi routine childcare occupations|L13.1 Routine sales and service occupations|L13.2 Routine production occupations|L13.3 Routine technical occupations|L13.4 Routine operative occupations|L13.5 Routine agricultural occupations":
                       ['Employers in large establishments', 'Higher managerial and administrative occupations',
                        'L3.1 Traditional employees', 'L3.2 New employees', 'L3.3 Traditional self-employed',
                        'L3.4 New self-employed', 'L4.1 Traditional employees', 'L4.2 New employees',
                        'L4.3 Traditional self-employed', 'L4.4 New self-employed', 'Lower managerial and administrative occupations',
                        'Higher supervisory occupations', 'L7.1 Intermediate clerical and administrative occupations',
                        'L7.2 Intermediate sales and service occupations', 'L7.3 Intermediate technical and auxiliary occupations',
                        'L7.4 Intermediate engineering occupations', 'L8.1 Employers in small establishments in industry, commerce, services etc.',
                        'L8.2 Employers in small establishments in agriculture', 'L9.1 Own account workers (non-professional)',
                        'L9.2 Own account workers (agriculture)', 'Lower supervisory occupations', 'L11.1 Lower technical craft occupations',
                        'L11.2 Lower technical process operative occupations', 'L12.1 Semi-routine sales occupations',
                        'L12.2 Semi-routine service occupations', 'L12.3 Semi-routine technical occupations', 'L12.4 Semi-routine operative occupations',
                        'L12.5 Semi-routine agricultural occupations', 'L12.6 Semi-routine clerical occupations', 'L12.7 Semi routine childcare occupations',
                        'L13.1 Routine sales and service occupations', 'L13.2 Routine production occupations', 'L13.3 Routine technical occupations',
                        'L13.4 Routine operative occupations', 'L13.5 Routine agricultural occupations'],
                   "Employers in large establishments|Higher managerial and administrative occupations|L3.1 ?Traditional? employees|L3.2 ?New? employees|L3.3 ?Traditional? self-employed|L3.4 ?New? self-employed|L4.1 ?Traditional? employees|L4.2 ?New? employees|L4.3 ?Traditional? self-employed|L4.4 ?New? self-employed|Lower managerial and administrative occupations|Higher supervisory occupations|L7.1 Intermediate clerical and administrative occupations|L7.2 Intermediate sales and service occupations|L7.3 Intermediate technical and auxiliary occupations|L7.4 Intermediate engineering occupations|L8.1 Employers in small establishments in industry, commerce, services etc.|L8.2 Employers in small establishments in agriculture|L9.1 Own account workers (non-professional)|L9.2 Own account workers (agriculture)|Lower supervisory occupations|L11.1 Lower technical craft occupations|L11.2 Lower technical process operative occupations|L12.1 Semi-routine sales occupations|L12.2 Semi-routine service occupations|L12.3 Semi-routine technical occupations|L12.4 Semi-routine operative occupations|L12.5 Semi-routine agricultural occupations|L12.6 Semi-routine clerical occupations|L12.7 Semi routine childcare occupations|L13.1 Routine sales and service occupations|L13.2 Routine production occupations|L13.3 Routine technical occupations|L13.4 Routine operative occupations|L13.5 Routine agricultural occupations":
                       ['Employers in large establishments', 'Higher managerial and administrative occupations',
                        'L3.1 Traditional employees', 'L3.2 New employees', 'L3.3 Traditional self-employed',
                        'L3.4 New self-employed', 'L4.1 Traditional employees', 'L4.2 New employees',
                        'L4.3 Traditional self-employed', 'L4.4 New self-employed', 'Lower managerial and administrative occupations',
                        'Higher supervisory occupations', 'L7.1 Intermediate clerical and administrative occupations',
                        'L7.2 Intermediate sales and service occupations', 'L7.3 Intermediate technical and auxiliary occupations',
                        'L7.4 Intermediate engineering occupations', 'L8.1 Employers in small establishments in industry, commerce, services etc.',
                        'L8.2 Employers in small establishments in agriculture', 'L9.1 Own account workers (non-professional)',
                        'L9.2 Own account workers (agriculture)', 'Lower supervisory occupations', 'L11.1 Lower technical craft occupations',
                        'L11.2 Lower technical process operative occupations', 'L12.1 Semi-routine sales occupations',
                        'L12.2 Semi-routine service occupations', 'L12.3 Semi-routine technical occupations', 'L12.4 Semi-routine operative occupations',
                        'L12.5 Semi-routine agricultural occupations', 'L12.6 Semi-routine clerical occupations', 'L12.7 Semi routine childcare occupations',
                        'L13.1 Routine sales and service occupations', 'L13.2 Routine production occupations', 'L13.3 Routine technical occupations',
                        'L13.4 Routine operative occupations', 'L13.5 Routine agricultural occupations'],
                   }

#                    "1.0|2.0":
#                        ["No","Yes"], # tryReduceImmigDKW4, achieveReduceImmigUKIPW4, achieveReduceImmigGrnW4, achieveReduceImmigDKW4, tryReduceInequalityDKW4, successReduceInequalityDKW4 # W4-5_comb # sharedContentOnline_1-5W4 W5_comb # voteMethodEurope_dkW2, discussantsAskedYouToVote_DKW2 ,discussantsAccompaniedVote_dkW2, referendumContact_dkW2 # W3_comb
 
rename_variable_dict = pd.DataFrame.from_dict( {k : "|".join(v) for k, v in rename_cat_dict.items()} , orient='index' ).reset_index()
rename_variable_dict.columns = ["original_cat_list","renameed_cat_list"]
rename_variable_dict.to_csv( BES_small_data_files + "rename_variable_dict.csv" )

def re_name(ques):
    if ques in rename_cat_dict.keys():
        return "|".join( rename_cat_dict[ques] )
    else:
        return ques

In [90]:
def is_ascii(s): # ascii or £
    return all( ( (ord(c) < 128) or (ord(c)==163))   for c in s)

def careful_not_ascii(s): # ascii or £
    if not isinstance(s,str):
        return False
    return not all( ( (ord(c) < 128) or (ord(c)==163))   for c in s)

def return_not_ascii(s): # ascii or £
    if not isinstance(s,str):
        return None
    return "|".join([ c for c in s if not ((ord(c) < 128) or (ord(c)==163)) ])

In [154]:
[x for x in Weasel_answers if careful_not_ascii(x)]

[]

In [152]:
[return_not_ascii(x) for x in Weasel_answers if careful_not_ascii(x)]

[]

In [153]:
Weasel_answers = ["Don't know",
                  "Prefer not to say", "Prefer not to answer", "Refused", "Unknown",
                  "Neither", "Other", "I would/will not vote", "Will not vote",
                  "I would not vote", "It depends", "Other",
                  "Don't follow politics on twitter",
                  "Yes, other", "Haven't thought about it",
                  "There wasn't a local election in my area", "No, haven't received it",
                  "I don't know what was negotiated", "I never received a response",
                  "There are not local elections in my area", "Can't remember",
                  "Varies too much to say", "Will not state a choice",
                  "All leaders equally good", "They are not eligible to vote",
                  "There are not local elections in my area", "Both/neither",
                  "Did not vote","Can't remember",
                  "Not sure"]

In [265]:
test = "Own outright|Own with a mortgage|Own (part-own) through shared ownership scheme (i.e. pay part mortgage, part rent)|Rent from a private landlord|Rent from my local authority|Rent from a housing association|Neither I live with my parents, family or friends but pay some rent to them|Neither I live rent-free with my parents, family or friends|Other"

In [266]:
test.split("|")

['Own outright',
 'Own with a mortgage',
 'Own (part-own) through shared ownership scheme (i.e. pay part mortgage, part rent)',
 'Rent from a private landlord',
 'Rent from my local authority',
 'Rent from a housing association',
 'Neither I live with my parents, family or friends but pay some rent to them',
 'Neither I live rent-free with my parents, family or friends',
 'Other']